In [1]:
import Queue
import threading
import urllib2
import time
import sys # for flushing print

In [2]:
class linkObject(object):
    link = [] # the actual link
    pos = 0
    parentPos = 0
    baconNumber = 0 # distance from source
    
    def __init__(self, l, p, pPos, bN):
        self.link = l
        self.pos = p
        self.parentPos = pPos
        self.baconNumber = bN

In [ ]:
class linkCounter(object):
    source = []
    target = []

    threads = []
    
    linkQueue = [] # holds the links to be checked
    linksSearched = [] # holds links already checked
    
    printQueue = []
    
    q = Queue.Queue() # holds the pages
    
    pagesFound = 0
    targetFound = False
    
    def __init__(self, s, t):
        self.source = s
        self.target = t
        
        # add source as first link object in queue
        self.linkQueue.append(linkObject(self.source, 0, 0, 0))
        #self.linkQueue.append(linkObject(self.target, 0, 0, 0))
        
    # returns true if the text is a link    
    def isLink(self, text):
        if len(text) > 7:
            if text[0] == 'h':
                if text[1] == 't':
                    if text[2] == 't':
                        if text[3] == 'p':
                            if text[4] == ':':
                                if text[5] == '/':
                                    if text[6] == '/':
                                        return True

        if len(text) > 8:
            if text[0] == 'h':
                if text[1] == 't':
                    if text[2] == 't':
                        if text[3] == 'p':
                            if text[4] == 's':
                                if text[5] == ':':
                                    if text[6] == '/':
                                        if text[7] == '/':
                                            return True

        if len(text) > 4:
            if text[0] == 'w':
                if text[1] == 'w':
                    if text[2] == 'w':
                        if text[3] == '.':
                            return True
    
        return False
    
    # gets all the html stuff from specified page
    def getInfo(self, l):
        #print "Getting:", ''.join(l.link)
        #sys.stdout.flush()
        
        try:
            fin = urllib2.urlopen(''.join(l.link))
            info = fin.read()
            fin.close()

            self.q.put(info)
            self.pagesFound += 1
            
            i = 0
            while i in range(len(info)):

                quote = []
                j = 1

                # if opening quote is found
                if info[i] == "\"":

                    # find everything within quote
                    # end if end of info reached or end quote is found
                    while i + j < len(info) and info[i + j] != "\"":
                        quote.append(info[i + j])

                        j += 1

                    # if its a link
                    if self.isLink(quote) == True:

                        if ''.join(quote) == self.target:
                            print "Target Found:", ''.join(quote)
                            print "Path:"
                            sys.stdout.flush()
                            
                            self.linksSearched.append(l)
                            self.linkQueue.remove(l)
            
                            self.targetFound = True
                            self.printPath(linkObject(quote, len(self.linkQueue), l.pos, l.baconNumber + 1))
                            return True
                            
                        # if link is not already on list
                        if self.linkAlreadyAdded(quote) == False:
                            self.linkQueue.append(linkObject(''.join(quote), len(self.linkQueue), l.pos, l.baconNumber + 1))
                            #print "Adding:", l.baconNumber + 1, ''.join(quote)
                            #sys.stdout.flush()

                i += j
                
            self.linksSearched.append(l)
            self.linkQueue.remove(l)
            
            return True
        
        except:
            print "Error opening:", ''.join(l.link)
            sys.stdout.flush()
            return False
        
    # checks to see if the link is already on either list
    def linkAlreadyAdded(self, link):
        for i in self.linksSearched:
            if i.link == link:
                return True
            
        for i in self.linkQueue:
            if i.link == link:
                return True
            
        return False
        
    def findParent(self, pPos):
        return 0
        #for i in self.linksSearched:
            #if i.pos == pPos:
                #return i
        
        #return 0
        
    def printPath(self, linkObj):
        if linkObj.baconNumber != 0:
            self.printPath(self.linksSearched[self.findParent(linkObj.parentPos)])
        
        print "->", ''.join(linkObj.link)
        
    def genPages(self):
        for i in self.linkQueue:
            t = threading.Thread(target = self.getInfo, args = (i,))
            t.daemon = True
            t.start()
            self.threads.append(t)
            
        # wait for all threads to finish before continuing to next level
        for i in self.threads:
            i.join()
    
    def findTarget(self):
        print "Starting at:", self.source
        print "Looking for:", self.target
        sys.stdout.flush()

        for i in range(2):
            print "Bacon:", i
            self.genPages()
            
        print "Links Searched"
        for i in self.linksSearched:
            print i.link
            
        print "Links not Searched but in Queue"
        for i in self.linkQueue:
            print i.link

In [ ]:
#test = linkCounter("http://www.reddit.com", "https://www.reddit.com/wiki/faq")
#test = linkCounter("http://www.reddit.com", "https://www.reddit.com/wiki/faq")
test = linkCounter("http://www.reddit.com", "https://www.reddit.com/wiki/faq")
test.findTarget()

Starting at: http://www.reddit.com
Looking for: https://www.reddit.com/wiki/faq
Bacon: 0
Bacon: